# Create the AKS cluster

In this notebook we'll setup the AKS cluster. To do so, we'll do the following:
1. check that there is enough quota to provision our desired cluster
2. provision the cluster using the `az cli`
3. set up blob fuse on the nodes so the pods in our kubernetes cluster can access our blob storage container

---

### Import packages and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import subprocess
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Provision AKS cluster and set up blobfuse

Set how many nodes you want to provision.

In [3]:
node_count = 3
set_key(env_path, "NODE_COUNT", str(node_count))

(True, 'NODE_COUNT', '3')

Check that there are enough core of the "Standard_NC6s_v3". If not, check that there are enough core of the "Standard_D2s_v3". If not, raise exception. 

In [4]:
vm_dict = {
    "NCSv3": {
        "size": "Standard_NC6s_v3",
        "cores": 6
    },
    "DSv3": {
        "size": "Standard_D2s_v3",
        "cores": 2
    }
}

print("Checking quota for family size DSv3...")
vm_family = "DSv3"
requested_cores = node_count * vm_dict[vm_family]["cores"]

def check_quota(vm_family):
    """
    returns quota object
    """
    results = subprocess.run([
        "az", "vm", "list-usage", 
        "--location", get_key(env_path, "REGION"), 
        "--query", "[?contains(localName, '%s')].{max:limit, current:currentValue}" % (vm_family)
    ], stdout=subprocess.PIPE)
    quota = json.loads(''.join(results.stdout.decode('utf-8')))
    return int(quota[0]['max']) - int(quota[0]['current'])

diff = check_quota(vm_family)
if diff <= requested_cores:
    print("Not enough cores of NCSv3 in region, asking for {} but have {}".format(requested_cores, diff))
    
    print("Retrying with family size DSv3...")
    vm_family = "DSv3"
    requested_cores = node_count * vm_dict[vm_family]["cores"]
    
    diff = check_quota(vm_family)
    if diff <= requested_cores:
        print("Not enough cores of DSv3 in region, asking for {} but have {}".format(requested_cores, diff))
        raise Exception("Core Limit", "Note enough cores to satisfy request")

print("There are enough cores, you may continue...") 

Checking quota for family size DSv3...
There are enough cores, you may continue...


Create the aks cluster. This step may take a while... Please note that this step creates another resource group in your subscription containing the actual compute of the AKS cluster.

*The `az aks create` command will generate service principal credentials (unless you explicitly specify it). So, if you have run this notebook before or have created an AKS cluster using the Azure CLI, you may need to clear service principal credentials stored to your machine's disk by running `rm ~/.azure/aksServicePrincipal.json`.*

In [5]:
!rm ~/.azure/aksServicePrincipal.json

In [6]:
!az aks create \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --node-count {node_count} \
    --node-vm-size {vm_dict[vm_family]["size"]} \
    --generate-ssh-keys

Finished service principal creation[##################################]  100.0000%Operation failed with status: 'Bad Request'. Details: Changing property 'servicePrincipalProfile.clientId' is not allowed.


In [7]:
crg = !az aks show --resource-group $RESOURCE_GROUP --name $AKS_CLUSTER --query nodeResourceGroup -o tsv
set_key(env_path, "CLUSTER_RESOURCE_GROUP", str(crg[0]))

(True, 'CLUSTER_RESOURCE_GROUP', 'MC_cambridge_cambridgecl_chinaeast2')

In [9]:
vm_list = !az vm list --resource-group {get_key(env_path, "CLUSTER_RESOURCE_GROUP")} -o table
!az vm list --resource-group {get_key(env_path, "CLUSTER_RESOURCE_GROUP")} -o table

Name                      ResourceGroup                        Location    Zones
------------------------  -----------------------------------  ----------  -------
aks-nodepool1-21996804-0  MC_cambridge_cambridgecl_chinaeast2  chinaeast2
aks-nodepool1-21996804-1  MC_cambridge_cambridgecl_chinaeast2  chinaeast2
aks-nodepool1-21996804-2  MC_cambridge_cambridgecl_chinaeast2  chinaeast2


In [10]:
set_key(env_path, "NODE_0", str(vm_list[2].split('  ')[0]))

(True, 'NODE_0', 'aks-nodepool1-21996804-0')

In [104]:
%%writefile fuser.sh

sudo apt update
sudo apt install git pkg-config libfuse-dev cmake libcurl4-gnutls-dev libgnutls28-dev libgcrypt20-dev -y
sudo apt install g++ -y
git clone https://github.com/pjh177787/azure-storage-fuse-cn
cd azure-storage-fuse-cn
./build.sh
stat /usr/bin/blobfuse
rm -f /usr/bin/blobfuse
cp ./build/blobfuse /usr/bin/
stat /usr/bin/blobfuse

Writing fuser.sh


In [106]:
!az vm run-command invoke \
    --resource-group {get_key(env_path, "CLUSTER_RESOURCE_GROUP")} \
    --name 'aks-nodepool1-21996804-0' \
    --command-id RunShellScript \
    --script @fuser.sh

{- Finished ..
  "value": [
    {
      "code": "ProvisioningState/succeeded",
      "displayStatus": "Provisioning succeeded",
      "level": "Info",
      "message": "Enable succeeded: \n[stdout]\ne-fuse-cn/build\nScanning dependencies of target blobfuse\n[  3%] Building CXX object CMakeFiles/blobfuse.dir/blobfuse/blobfuse.cpp.o\n[  6%] Building CXX object CMakeFiles/blobfuse.dir/blobfuse/fileapis.cpp.o\n[  9%] Building CXX object CMakeFiles/blobfuse.dir/blobfuse/directoryapis.cpp.o\n[ 12%] Building CXX object CMakeFiles/blobfuse.dir/blobfuse/utilities.cpp.o\n[ 15%] Building CXX object CMakeFiles/blobfuse.dir/azure-storage-cpp-lite/src/base64.cpp.o\n[ 18%] Building CXX object CMakeFiles/blobfuse.dir/azure-storage-cpp-lite/src/constants.cpp.o\n[ 21%] Building CXX object CMakeFiles/blobfuse.dir/azure-storage-cpp-lite/src/hash.cpp.o\n[ 24%] Building CXX object CMakeFiles/blobfuse.dir/azure-storage-cpp-lite/src/utility.cpp.o\n[ 27%] Building CXX object CMakeFiles/blobfuse.dir/azure-stora

In [107]:
!az vm run-command invoke \
    --resource-group {get_key(env_path, "CLUSTER_RESOURCE_GROUP")} \
    --name 'aks-nodepool1-21996804-0' \
    --command-id RunShellScript \
    --script 'stat /usr/bin/blobfuse'

{- Finished ..
  "value": [
    {
      "code": "ProvisioningState/succeeded",
      "displayStatus": "Provisioning succeeded",
      "level": "Info",
      "message": "Enable succeeded: \n[stdout]\n  File: '/usr/bin/blobfuse'\n  Size: 1138560   \tBlocks: 2224       IO Block: 4096   regular file\nDevice: 801h/2049d\tInode: 3487        Links: 1\nAccess: (0755/-rwxr-xr-x)  Uid: (    0/    root)   Gid: (    0/    root)\nAccess: 2019-07-18 09:38:56.281227953 +0000\nModify: 2019-07-18 09:38:56.281227953 +0000\nChange: 2019-07-18 09:38:56.281227953 +0000\n Birth: -\n\n[stderr]\n",
      "time": null
    }
  ]
}


In [95]:
!kubectl get nodes -o wide

NAME                       STATUS   ROLES   AGE    VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-21996804-0   Ready    agent   5h9m   v1.12.7   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1040-azure   docker://3.0.4
aks-nodepool1-21996804-1   Ready    agent   5h9m   v1.12.7   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1040-azure   docker://3.0.4
aks-nodepool1-21996804-2   Ready    agent   5h9m   v1.12.7   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1040-azure   docker://3.0.4


In [96]:
!kubectl get po --namespace=kube-system -o wide

NAME                                    READY   STATUS    RESTARTS   AGE     IP           NODE                       NOMINATED NODE
blobfuse-flexvol-installer-f7jjz        1/1     Running   0          74m     10.244.0.9   aks-nodepool1-21996804-2   <none>
blobfuse-flexvol-installer-fzjlm        1/1     Running   0          74m     10.244.1.3   aks-nodepool1-21996804-1   <none>
blobfuse-flexvol-installer-nkcf9        1/1     Running   0          74m     10.244.2.4   aks-nodepool1-21996804-0   <none>
coredns-66d44cdc7c-54wn5                1/1     Running   0          5h12m   10.244.0.2   aks-nodepool1-21996804-2   <none>
coredns-66d44cdc7c-lh5xc                1/1     Running   0          5h8m    10.244.2.2   aks-nodepool1-21996804-0   <none>
coredns-autoscaler-6fdbf9f665-5vkjl     1/1     Running   0          5h12m   10.244.0.7   aks-nodepool1-21996804-2   <none>
heapster-5f4dfbc58d-zgh2q               2/2     Running   0          5h12m   10.244.0.4   aks-nodepool1-21996804-2   <none>


Install Kubectl - this tool is used to manage the kubernetes cluster.

In [115]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


In [16]:
!az aks get-credentials \
    --resource-group {get_key(env_path, 'RESOURCE_GROUP')}\
    --name {get_key(env_path, 'AKS_CLUSTER')}

Merged "cambridgecl" as current context in /home/aperture/.kube/config


Check also that the nodes are up and ready using this command. You may choose to run this command in a new cell.
```bash
!kubectl get nodes
```

### Blobfuse on AKS

Now we setup our AKS cluster so that we have blob storage mounted onto the nodes using blob fuse. More info [here](https://github.com/Azure/kubernetes-volume-drivers/tree/master/flexvolume/blobfuse).

Install blobfuse driver on every agent VM.

In [18]:
!sudo docker login --username pjh177787 --password 'Hans&951022'

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/aperture/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [89]:
!kubectl create -f https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml

Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml": daemonsets.apps "blobfuse-flexvol-installer" already exists


Check daemonset status.

In [90]:
!kubectl describe daemonset blobfuse-flexvol-installer --namespace=kube-system

Name:           blobfuse-flexvol-installer
Selector:       name=blobfuse
Node-Selector:  beta.kubernetes.io/os=linux
Labels:         k8s-app=blobfuse
Annotations:    deprecated.daemonset.template.generation: 1
Desired Number of Nodes Scheduled: 3
Current Number of Nodes Scheduled: 3
Number of Nodes Scheduled with Up-to-date Pods: 3
Number of Nodes Scheduled with Available Pods: 3
Number of Nodes Misscheduled: 0
Pods Status:  3 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  name=blobfuse
  Containers:
   blobfuse-flexvol-installer:
    Image:        mcr.microsoft.com/k8s/flexvolume/blobfuse-flexvolume:1.0.9
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:
      /etc/kubernetes/volumeplugins/ from volplugins (rw)
      /var/log/ from varlog (rw)
  Volumes:
   varlog:
    Type:          HostPath (bare host directory volume)
    Path:          /var/log/
    HostPathType:  
   volplugins:
    Type:          HostPath (bare host dir

In [91]:
!kubectl get po --namespace=kube-system -o wide

NAME                                    READY   STATUS    RESTARTS   AGE     IP           NODE                       NOMINATED NODE
blobfuse-flexvol-installer-f7jjz        1/1     Running   0          34m     10.244.0.9   aks-nodepool1-21996804-2   <none>
blobfuse-flexvol-installer-fzjlm        1/1     Running   0          34m     10.244.1.3   aks-nodepool1-21996804-1   <none>
blobfuse-flexvol-installer-nkcf9        1/1     Running   0          34m     10.244.2.4   aks-nodepool1-21996804-0   <none>
coredns-66d44cdc7c-54wn5                1/1     Running   0          4h33m   10.244.0.2   aks-nodepool1-21996804-2   <none>
coredns-66d44cdc7c-lh5xc                1/1     Running   0          4h29m   10.244.2.2   aks-nodepool1-21996804-0   <none>
coredns-autoscaler-6fdbf9f665-5vkjl     1/1     Running   0          4h33m   10.244.0.7   aks-nodepool1-21996804-2   <none>
heapster-5f4dfbc58d-zgh2q               2/2     Running   0          4h33m   10.244.0.4   aks-nodepool1-21996804-2   <none>


Set up credentials for blobfuse.

In [92]:
!kubectl create secret generic blobfusecreds \
    --from-literal accountname={get_key(env_path, 'STORAGE_ACCOUNT_NAME')} \
    --from-literal accountkey={get_key(env_path, 'STORAGE_ACCOUNT_KEY')} \
    --type="azure/blobfuse"

secret/blobfusecreds created


Set the mount directory on our AKS cluster as en dotenv variable.

In [93]:
set_key(env_path, "MOUNT_DIR", "/data")

(True, 'MOUNT_DIR', '/data')

Continue to the next [notebook](/notebooks/04_style_transfer_on_aks.ipynb).